In [87]:
from wfield import *
import os 
import h5py
import pandas as pd
from scipy.io import loadmat
from scipy.signal import savgol_filter

In [115]:
directory = "D:/Behavior/Animals/"
animal = "FV-4/"
maze = "SD/"
session = "12-Jul-2022/"
file = "{}/"
sync = "sync"
path_sync = os.path.join(directory, animal, maze, session, sync)
os.chdir(path_sync)
print(path_sync)

D:/Behavior/Animals/FV-4/SD/12-Jul-2022/sync


In [116]:
filename = "FV_4_SD_2022-07-12_0001.h5"
h5 = h5py.File(filename,'r')

#extract data from h5 file for visualization
with h5py.File(filename, "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = str(list(f.keys())[1])
    
    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_analog = f[a_group_key][data[0]][()]  # returns as a numpy array
    ds_digital = f[a_group_key][data[1]][()]
    ds_time = f[a_group_key][data[2]][()]

Keys: <KeysViewHDF5 ['header', 'sweep_0001']>
<class 'h5py._hl.group.Group'>


In [117]:
#load mat files with trial start times (in wav units) 

imaging_spk = loadmat('imaging_spk.mat')
times = loadmat('times.mat')
correct = loadmat('correct.mat')

In [118]:
#extract start and end iterations for each trial 

mat = loadmat('imaging_spk.mat') # insert your filename here
mat_load = [[element for element in upperElement] for upperElement in mat['imaging_spk']] # insert your fieldname here

Start_it = []
End_it = []
num_trials = 272 #figure out a way to not have to hard code this 
for i in list(range(num_trials)):
    trial = mat_load[0][i]
    start_it = trial[0][0][0]
    Start_it.append(start_it)
    end_it = trial[1][0][0]
    End_it.append(end_it)

In [119]:
#extract correct/incorrect info 

mat = loadmat('correct.mat') # insert your filename here
mat_load = [[element for element in upperElement] for upperElement in mat['correct']] # insert your fieldname here
correct_val = [] #array with the correct/incorrect info 
num_trials = 275 #figure out a way to not have to hard code this 
for i in list(range(num_trials)):
    correct_val.append(mat_load[i][0][0][0])

In [120]:
#extract start and end wav time points 

mat = loadmat('times.mat') # insert your filename here
mat_load = [[element for element in upperElement] for upperElement in mat['times']] # insert your fieldname here

wav_start_points = []
for i in Start_it[:250]: #will make this for all trials once Noelle fixes bug in extraction code 
    wav_start = mat_load[i][0]
    wav_start_points.append(wav_start)

wav_end_points = []
for i in End_it[:250]: #will make this for all trials once Noelle fixes bug in extraction code 
    wav_end = mat_load[i][0]
    wav_end_points.append(wav_end)

In [121]:
#find when imaging started 
a = ds_digital[0][:]
#img_start = np.where(a==1)[0][0] #start of imaging 
start_stop_wav_points = np.where(np.diff(a) >= 1)[0] #values of imaging start and stop points 

In [122]:
#get image start times (in wav)
img_start = start_stop_wav_points[::2]
img_end = start_stop_wav_points[1::2]
#img_start = img_start[0]

In [123]:
last_trial_insession =[]
for i in list(range(8)):
    vals = np.where(wav_end_points < img_end[i])[0]
    last_trial_insession.append(vals[-1])

In [124]:
def wav_point(trial):
    start = wav_start_points[trial]
    end = wav_end_points[trial]
    return start, end

def frames_from_wav(start,end,img_start):
    length_from_img_start_to_startoftrial = (start - img_start)/10000
    length_from_img_start_to_endoftrial = (end- img_start)/10000
    start_int = int(length_from_img_start_to_startoftrial*30)
    end_int = int(length_from_img_start_to_endoftrial*30)
    return start_int, end_int

In [125]:
def extract_times(img_session):
    path = os.path.join(directory, animal, maze, session, file.format(img_session+1))
    os.chdir(path)
    print(path)
    if img_session == 0: 
        trial_in_img_session = list(range(last_trial_insession[0]))
    elif img_session > 0:
        x = last_trial_insession[img_session]
        y = last_trial_insession[img_session-1]
        trial_in_img_session = list(range(y,x))
    
    start_frames = []
    end_frames = []

    for trial in trial_in_img_session:
        start_frame = frames_from_wav(wav_point(trial)[0],wav_point(trial)[1], img_start[img_session])[0]
        end_frame = frames_from_wav(wav_point(trial)[0],wav_point(trial)[1], img_start[img_session])[1]
        start_frames.append(start_frame)
        end_frames.append(end_frame)
    df = pd.DataFrame(trial_in_img_session, columns=['trial'])
    df['trial'] = df['trial']+1
    df['img session'] = img_session+1
    df['start frame'] = start_frames
    df['end frame'] = end_frames 
    return df

In [126]:
All_data = pd.concat([extract_times(0),
           extract_times(1),
           extract_times(2),
           extract_times(3),
           extract_times(4),
           extract_times(5),
           extract_times(6),
           extract_times(7)
            ])

D:/Behavior/Animals/FV-4/SD/12-Jul-2022/1/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/2/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/3/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/4/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/5/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/6/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/7/
D:/Behavior/Animals/FV-4/SD/12-Jul-2022/8/


In [127]:
All_data['correct'] = correct_val[:len(All_data.index)]

In [128]:
path_sync = os.path.join(directory, animal, maze, session, sync)
os.chdir(path_sync)
print(path_sync)

All_data
All_data.to_csv("All_data.csv")

D:/Behavior/Animals/FV-4/SD/12-Jul-2022/sync


In [114]:
All_data

,trial,img session,start frame,end frame,correct
0,1,1,-56,236,1
1,2,1,335,485,0
2,3,1,806,1072,1
3,4,1,1172,1365,1
4,5,1,1460,1649,1
...,...,...,...,...,...
25,226,8,7030,7200,1
26,227,8,7294,7488,0
27,228,8,7809,7967,1
28,229,8,8061,8243,1
